In [1]:
import cv2
import CV_utils as cvu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.cluster as sc
import sklearn.mixture as sm
import multiprocessing
import time
%matplotlib inline

# READ FRAMES

In [ ]:
l_b_frames, l_bgs_frames, l_c_frames, l_gs_frames = cvu.read_frames('Video/representative/Lightness/Video attached to the ICMI 2017 Paper - Lightness.mp4')
f_b_frames, f_bgs_frames, f_c_frames, f_gs_frames = cvu.read_frames('Video/representative/Fragility/Video attached to the ICMI 2017 Paper - Fragility.mp4')

# CORNERS TRACKING

In [ ]:
cl1 = sc.KMeans(n_clusters=10)
cl2 = sc.KMeans(n_clusters=5)

for i, binary_frame in enumerate(l_b_frames):
    gray = np.float32(binary_frame)
    dst = cv2.cornerHarris(gray,2,3,0.04)

    #result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # Threshold for an optimal value, it may vary depending on the image.
    colored_frame = l_c_frames[i]
    
    #clustering
    dst = np.uint8(dst)
    dst_ind = np.argwhere(dst>0)
    centroids = cl1.fit(dst_ind).cluster_centers_
                    
    centroids = cl2.fit_predict(centroids)

    cvu.draw_centroids(centroids, colored_frame)
        
    cv2.imshow('centroids',colored_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# CLUSTERING METHODS

## CENTROIDS FINDING TESTS

### KMEANS CLUSTERING OVER WHITE PIXELS

In [ ]:
cl3 = sc.KMeans(n_clusters=9)

white_pixels = np.argwhere(l_b_frames[0] == 255)

labels = cl3.fit_predict(white_pixels)

coloured_frame = l_c_frames[0]

cvu.draw_clusters(labels, white_pixels, coloured_frame)

plt.imshow(coloured_frame)

In [ ]:
def key_point_mass_kmeans(binary_frame_list, colored_frame_list):
    
    cl3 = sc.KMeans(n_clusters=20)

    for i, frame in enumerate(binary_frame_list):
        
        white_pixel = np.argwhere(frame == 255)

        centroids = cl3.fit(white_pixel).cluster_centers_

        colored_frame = cvu.draw_centroids(centroids, colored_frame_list[i])

        cv2.imshow('blob',colored_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

    cv2.destroyAllWindows()

In [ ]:
# key_point_mass_kmeans(f_b_frames, f_c_frames)
key_point_mass_kmeans(l_b_frames, l_c_frames)

### KMEANS CLUSTERING OVER GRAYFRAME SHAPE (less reliable than the white-pixels version)

In [ ]:
cl3 = sc.KMeans(n_clusters=9)

gs_shape = np.argwhere(l_bgs_frames[0] != 0)

labels = cl3.fit_predict(gs_shape)

coloured_frame = l_c_frames[0]

cvu.draw_clusters(labels, gs_shape, coloured_frame)

plt.imshow(coloured_frame)

In [ ]:
def key_point_mass_kmeans_with_gsShape(binarygray_frame_list, colored_frame_list):
    
    cl = sc.KMeans(n_clusters=20)

    for i, bg_frame in enumerate(binarygray_frame_list):
        
        gs_shape = np.argwhere(bg_frame != 0)

        centroids = cl.fit(gs_shape).cluster_centers_

        coloured_frame = cvu.draw_centroids(centroids, colored_frame_list[i])

        cv2.imshow('blob', coloured_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

    cv2.destroyAllWindows()

In [ ]:
# key_point_mass_kmeans_with_gsShape(f_bgs_frames, f_c_frames)
key_point_mass_kmeans_with_gsShape(l_bgs_frames, l_c_frames)

### COMPARE WHITE-PIXEL WITH GRAYSCALE SHAPE K-MEANS CLUSTERING

In [ ]:
def compare_kmeans(binary_frame_list, binarygray_frame_list, colored_frame_list):
    
    cl_b = sc.KMeans(n_clusters=20)
    cl_gs = sc.KMeans(n_clusters=20)

    for i, (bingray_frame, binary_frame) in enumerate(zip(binarygray_frame_list, binary_frame_list)):
        
        white_pixel_gs = np.argwhere(bingray_frame != 0)
        white_pixel_b = np.argwhere(binary_frame == 255)

        centroids_gs = cl_gs.fit(white_pixel_gs).cluster_centers_
        centroids_b = cl_b.fit(white_pixel_b).cluster_centers_
        
        colored_frame = cvu.draw_centroids(centroids_gs, colored_frame_list[i])
        colored_frame = cvu.draw_centroids(centroids_b, colored_frame_list[i], color=[255,0,0])
        
        cv2.imshow('blob',colored_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

    cv2.destroyAllWindows()

In [ ]:
compare_kmeans(l_b_frames, l_bgs_frames, l_c_frames)

### NESTED CLUSTERING OVER CENTROIDS (high centroids movement on adaption)

In [ ]:
def nested_clustering(binary_frames, coloured_frames):
    cl1 = sc.KMeans(n_clusters=20)
    cl2 = sc.KMeans(n_clusters=10)

    for binary_frame, c_frame in zip(binary_frames, coloured_frames):
        
        white_pixel_b = np.argwhere(binary_frame == 255)

        centroids_b = cl1.fit(white_pixel_b).cluster_centers_
        
        cvu.draw_centroids(centroids_b, c_frame, color=[255,0,0])

        centroids = cl2.fit(centroids_b).cluster_centers_
        
        cvu.draw_centroids(centroids, c_frame)

        cv2.imshow('centroids', c_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [ ]:
nested_clustering(l_b_frames, l_c_frames)

### GAUSSIAN CLUSTERING OVER WHITE PIXELS

In [ ]:
gm = sm.GaussianMixture(n_components=8)
white_pixels = np.argwhere(l_b_frames[0] == 255)

gm.fit(white_pixels)
labels = gm.predict(white_pixels)

coloured_frame = l_c_frames[0]

cvu.draw_clusters(labels, white_pixels, coloured_frame)

plt.imshow(coloured_frame)

In [ ]:
def key_point_mass_Gaussian(binary_frame_list, colored_frame_list):
    
    gm = sm.GaussianMixture(n_components=20)

    for i, frame in enumerate(binary_frame_list):
        white_pixel = np.argwhere(frame == 255)

        centroids = gm.fit(white_pixel).means_

        colored_frame = cvu.draw_centroids(centroids, colored_frame_list[i])

        cv2.imshow('blob',colored_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

    cv2.destroyAllWindows()

In [ ]:
key_point_mass_Gaussian(l_b_frames, l_c_frames)
# key_point_mass_Gaussian(f_b_frames, f_c_frames)

### HIERARCHICAL CLUSTERING OVER WHITE PIXELS (computationally too expensive)

In [ ]:
hc = sc.AgglomerativeClustering(n_clusters=9)

white_pixels = np.argwhere(l_b_frames[0] == 255)

labels = hc.fit_predict(white_pixels)

coloured_frame = l_c_frames[0]

cvu.draw_clusters(labels, white_pixels, coloured_frame)

plt.imshow(coloured_frame)

In [ ]:
def key_point_mass_Hierarchical(binary_frame_list, colored_frame_list):
    
    hc = sc.AgglomerativeClustering(n_clusters=9)

    for i, frame in enumerate(binary_frame_list):
        white_pixel = np.argwhere(frame == 255)

        labels = hc.fit_predict(white_pixel)

        centroids = []

        #calculating centroids
        for l in np.unique(labels):
            cluster = white_pixel[labels == l]
            centroids.append(np.mean(cluster))

        colored_frame = cvu.draw_centroids(centroids, colored_frame_list[i])

        cv2.imshow('blob', colored_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

    cv2.destroyAllWindows()

In [ ]:
key_point_mass_Hierarchical(l_b_frames, l_c_frames)

### SPECTRAL CLUSTERING OVER WHITE PIXELS (computationally too expensive)

In [ ]:
cl = sc.SpectralClustering(n_clusters=6)

white_pixels = np.argwhere(l_b_frames[0] == 255)

labels = cl.fit_predict(white_pixels)

coloured_frame = l_c_frames[0]

cvu.draw_clusters(labels, white_pixels, coloured_frame)

plt.imshow(coloured_frame)

## CENTROIDS TRACKING OVER TIME (euclidean distance)

In [2]:
def get_related_centroid(first_centroids, second_centroids, n_clusters):
    
    from sklearn.metrics.pairwise import euclidean_distances
        
    # compute the pairwise euclidean distance between the found centroids
    distance = euclidean_distances(first_centroids, second_centroids)
    
    ordered_idx = []
    distances = []
    
    # order the flattened distances matrix 
    for idx_value in np.argsort(distance, axis=None):
        # row, col will be obtained by the quotient and the remainder of the division by n_cluster
        q, r = divmod(idx_value, n_clusters)
        ordered_idx.append((q,r))
        distances.append(distance[q,r])

    f_idx = set(range(n_clusters))
    s_idx = set(range(n_clusters))
    
    closest_idx = {}
    
    for row, col in ordered_idx:
        # if empty, then already seen all centroids
        if len(f_idx)==0: break
        # already found
        if (row not in f_idx) or (col not in s_idx): continue
            
        # old_c -> new_c, diff
        closest_idx[str(row)] = [col, second_centroids[col]-first_centroids[row]]
        
        f_idx.remove(row)
        s_idx.remove(col)

    return closest_idx, np.sum(distances[:(n_clusters//2)])/(n_clusters//2)

In [3]:
def cluster_over_stdDirections(associations, n_clusters, mean_distance):
    
    projections = np.zeros((n_clusters, len(cvu.ref_angles)))
    
    for map_idx in range(n_clusters):
        # index that follow a centroid evolution
        next_centr = str(map_idx)
        
        # move along all the frames pairs (dictionaries: old->new)
        for table in associations:
            proj_counter = np.zeros(len(cvu.ref_angles))
            
            # evaluate the projection on the reference angles (defined in CV_utils.py) 
            for h in range(len(cvu.ref_angles)//2):

                # project
                proj_counter[h] = np.dot(table[next_centr][1], cvu.ref_angles[h])
                
                # if negative it's the origin-symmetric angle
                if proj_counter[h] < 0:
                    proj_counter[h+(len(cvu.ref_angles)//2)] -= proj_counter[h]
                    proj_counter[h] = 0
                    
            #to zero all the projections but the maximum one -- clustering    
            max_index = np.argmax(proj_counter)
            proj_counter[[idx for idx in range(len(cvu.ref_angles)) if idx!=max_index]] = 0
            # since the evolution tracking of the centroids is computed purely on geometric distances (sorted by size)
            # some centroid will be linked to perhaps a totally different centroid (this may happen when the 
            # dancer's head disappear for instance), try to patch this scenario with a fixed distance value
            # (possibly the distance average value of the closer centroid)
            if proj_counter[max_index] > mean_distance: proj_counter[max_index] = mean_distance
        
            projections[map_idx] += np.square(proj_counter)
            
            next_centr = str(table[next_centr][0])
            
    return projections/len(associations)

In [18]:
def kMeans_clustering_over_video(return_dict, p_idx, binary_frames, n_clusters=20, verbose=False):
    
    cl = sc.KMeans(n_clusters=n_clusters, random_state=42)
        
    associations = []
    mean_distances = []
        
    former_centr = cl.fit(np.argwhere(binary_frames[0]==255)).cluster_centers_
    skip = 1
                
    for f_idx, binary_frame in enumerate(binary_frames[1:]):
        
        if verbose: start = time.time()
            
        # skip consecutive frames, pick one every n_skip
        #if skip != 0 and skip <= 3:
         #   skip += 1
          #  continue
        #else:
         #   skip = 0
        
        if verbose: print('\t training on frames pair {}-{} on a total of {} frames'.format(f_idx, f_idx+1, len(binary_frames)))
        
        latter_centr = cl.fit(np.argwhere(binary_frames[f_idx+1]==255)).cluster_centers_
        
        association, mean_distance = get_related_centroid(former_centr, latter_centr, n_clusters)
        associations.append(association)
        mean_distances.append(mean_distance)

        former_centr = latter_centr
        
        skip += 1
        
        if verbose: print('done in %s' % (time.time()-start))
    
    return_dict[p_idx] = cluster_over_stdDirections(associations, n_clusters, np.sum(mean_distances)/len(mean_distances))


### table creation

In [15]:
def load_and_create_centroid (path_to_read_binary, path_to_write, n_centroids=20, limit_videos=0, 
                              limit_frames=4, verbose=False):
    
    for j in range(1,16):
        
        res = []
        
        print("Extracting data for the videos from {} to {}".format(((j-1)*10)+1, j*10))
        
        video_frames_list = cvu.binary_read(path_to_read_binary, n=j, limit_videos=limit_videos, verbose=verbose)
        
        if limit_videos == 0: lv=len(video_frames_list)
        else: lv = limit_videos
            
        # PARRALELISATION INITIALISATION
        jobs = []
        manager = multiprocessing.Manager()
        return_dict = manager.dict()
        
        # ----------- PARALLEL SEGMENT
        for i, video_frames in enumerate(video_frames_list):

            if limit_frames==0: lf = len(video_frames)
            else: lf = limit_frames

            video_frames = cvu.reduce_video_len(video_frames, n_frames=lf)
                        
            process = multiprocessing.Process(target=kMeans_clustering_over_video, args=[return_dict, i, video_frames, n_centroids, verbose])
            jobs.append(process)
            process.start()
        # ----------- END PARALLEL SEGMENT
            
        for i, p in enumerate(jobs):
            p.join()
            
        for i, p in enumerate(jobs):
            res.append(return_dict[i].flatten())
        
        pd.DataFrame(data=res).to_csv(path_to_write+'centroids_clustering_over{}_{}.csv'.format(limit_frames, j))


In [17]:
load_and_create_centroid('Video/binary_video/', 'datasets/centrTracking/', verbose=True)

Extracting data for the videos from 1 to 10
converting 20170113_t001_segm01.mpeg : 1 out of 10
converting 20170113_t002_segm01.mpeg : 2 out of 10
converting 20170113_t003_segm01.mpeg : 3 out of 10
converting 20170113_t005_segm01.mpeg : 4 out of 10
converting 20170113_t006_segm01.mpeg : 5 out of 10
converting 20170113_t007_segm01.mpeg : 6 out of 10
converting 20170113_t008_segm01.mpeg : 7 out of 10
converting 20170113_t009_segm01.mpeg : 8 out of 10
converting 20170113_t010_segm01.mpeg : 9 out of 10
converting 20170113_t011_segm01.mpeg : 10 out of 10
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on frames pair -2-1 on a total of 4 frames
	 training on

done in 13.749013185501099
	 training on frames pair 0-3 on a total of 4 frames
done in 9.185946941375732
done in 15.023051023483276
	 training on frames pair 0-3 on a total of 4 frames
done in 14.233999967575073
done in 8.78127121925354
done in 13.104721069335938
	 training on frames pair 0-3 on a total of 4 frames
done in 10.008743047714233
done in 18.341919898986816
	 training on frames pair 0-3 on a total of 4 frames
done in 7.7028539180755615
done in 15.81414794921875
	 training on frames pair 0-3 on a total of 4 frames
done in 9.849924802780151
done in 8.252512693405151
done in 9.358768939971924
done in 6.276507139205933
done in 5.0628180503845215
Extracting data for the videos from 31 to 40
converting 20170114_t040_segm01.mpeg : 1 out of 10
converting 20170114_t041_segm01.mpeg : 2 out of 10
converting 20170114_t042_segm01.mpeg : 3 out of 10
converting 20170114_t042_segm02.mpeg : 4 out of 10
converting 20170114_t043_segm01.mpeg : 5 out of 10
converting 20170114_t044_segm01.mpeg :

KeyboardInterrupt: 

### RLS over CENTROID TRACKING

In [2]:
X = pd.read_csv('datasets/centroids_clustering_over100.csv', index_col=None)#0)
y = cvu.load_marks()[1]
y_l = y.values[:,0]
y_f = y.values[:,1]

In [3]:
X_tr, y_l_tr, y_f_tr, X_ts, y_l_ts, y_f_ts = cvu.random_sampling(X.values, y_l, y_f)

RLS

In [4]:
alphas = np.arange(0.001, 1, 0.007)

In [5]:
# LIGHTNESS
cvu.print_results('RLS',cvu.rlsCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts))

Model: RLS
	regularizer alpha: 0.995
	mean error: 1.1063062362624956
	variance of the error: 0.9561799769581893
	coefficient w:
[-0.08696062  0.27875793  0.20441986  0.24748354 -0.02472268  0.19222954
  0.20772826  0.11535959  0.20826743  0.36613765 -0.37364226  0.14722876
  0.19231589  0.20956967  0.05767177 -0.19297894 -0.03213832  0.19805577
  0.08727244  0.15096401  0.02332088 -0.15220475 -0.08037439  0.01240257
  0.08717078  0.01081448 -0.33919483  0.06825607  0.09202614  0.23307957
 -0.18613445 -0.3144973   0.11963445  0.49314889 -0.01359083  0.27156067
  0.06804062  0.00668052  0.25078651  0.17121375 -0.06825924  0.01426705
  0.26380184  0.01930638  0.19153358 -0.22384849  0.02495429  0.22987837
 -0.00180769 -0.24673558 -0.32443728  0.33351142 -0.32686538 -0.0119978
  0.1198952   0.51635561 -0.03698162  0.42347381  0.01056622 -0.27369921
 -0.0843951  -0.54749212 -0.35809198  0.22126217 -0.05261709  0.18962148
 -0.29505249 -0.22175841  0.11411141 -0.17200876 -0.12560939 -0.039549

In [6]:
# FRAGILITY
cvu.print_results('RLS', cvu.rlsCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts))

Model: RLS
	regularizer alpha: 0.995
	mean error: 1.1935342976264065
	variance of the error: 0.9646553161956625
	coefficient w:
[ 0.05235343 -0.16622489  0.1295035  -0.15551152 -0.14891185 -0.07815401
 -0.14684368 -0.05381465 -0.48083645 -0.17855847  0.12905513  0.40580365
 -0.01604797 -0.46068768 -0.14519515  0.18318001  0.30460946 -0.12214555
  0.08455595 -0.30895464 -0.46150833  0.23547587  0.41925673 -0.01074456
 -0.25853983  0.06116082  0.04929415 -0.38343644 -0.0611208  -0.11226291
  0.4396265   0.00715924  0.35903543 -0.09774982  0.08520815 -0.4007364
 -0.306144   -0.24857141 -0.07675982 -0.42090559 -0.04527686  0.55632676
 -0.36016798 -0.04336974  0.08392373  0.37529294 -0.06234452 -0.36925601
  0.14502819 -0.18074697 -0.20245525 -0.44889021  0.8546259  -0.14882103
 -0.11412293  0.10716885  0.00840083  0.2128954  -0.03341787 -0.14221537
  0.18609423  0.46137057  0.19556718 -0.32056341 -0.20009134  0.1098705
  0.11040219  0.12131772  0.0479488  -0.08271644  0.00443616  0.0079042

LASSO

In [7]:
alphas = np.arange(0.001, 1, 0.007)

In [8]:
#LIGHTNESS
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts))

Model: LASSO
	regularizer alpha: 0.092
	mean error: 0.6589588143587017
	variance of the error: 0.19131651897352492
	coefficient w:
[-0.         -0.          0.         -0.         -0.         -0.
  0.          0.          0.          0.         -0.         -0.
  0.          0.         -0.         -0.          0.         -0.
 -0.         -0.          0.         -0.          0.         -0.
 -0.          0.          0.          0.         -0.04659406 -0.
  0.          0.          0.          0.         -0.         -0.
  0.          0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.          0.          0.
 -0.         -0.         -0.          0.         -0.         -0.
  0.          0.05604347  0.          0.         -0.         -0.
 -0.         -0.          0.         -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.          0.          0.         -0.07576842
  0.          0.

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
#FRAGILITY
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts))

Model: LASSO
	regularizer alpha: 0.148
	mean error: 0.8446709781823347
	variance of the error: 0.24564075700338517
	coefficient w:
[-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.00494228 -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.          0.         -0.
 -0.11071019 -0.         -0.          0.         -0.         -0.
 -0.          0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.02324152 -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.        

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


KERNEL

In [26]:
alphas = {'alpha': np.arange(0.01, 10, 0.7),'degree':range(3,10)}
k1 = 'poly'
k2 = 'sigmoid'

In [27]:
#LIGHTNESS - poly
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts, kernel=k1))

Model: RIDGE KERNEL
	regularizer alpha: 6.31
	mean error: 0.6788680151141095
	variance of the error: 0.2438436154823583
	coefficient w:
[ 0.02828927 -0.03909785  0.03053804 -0.05836288  0.08207344  0.00610336
 -0.06243693  0.07551616 -0.10302943 -0.03180566  0.07762458 -0.01456226
  0.04708739  0.13464499  0.21857338 -0.03580314 -0.02366532  0.00751151
 -0.02593455 -0.01422534  0.2534302   0.12324842 -0.00463337 -0.03054365
  0.16026476  0.10970633 -0.03837178  0.08813683 -0.17394349 -0.04656924
 -0.11410814  0.05497203 -0.07943224 -0.13171103 -0.02091847 -0.18138402
 -0.03829277 -0.12276409 -0.02113168  0.11574249 -0.1319848   0.04726852
 -0.07227556  0.1175583  -0.01575829  0.28406791 -0.10952598  0.17838985
  0.07848906 -0.00877179  0.03351943  0.27534604 -0.10189103  0.00142753
  0.0661582  -0.03511189 -0.05725541 -0.01365932  0.22543533 -0.15434668
  0.06993005  0.02068097  0.09230162 -0.03765378  0.1173586  -0.15520871
 -0.0316687  -0.08402437 -0.02397037  0.00586109 -0.08199195 

In [28]:
#LIGHTNESS - sigmoid
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_l_tr, y_l_ts, kernel=k2))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154

Model: RIDGE KERNEL
	regularizer alpha: 2.1099999999999994
	mean error: 0.6637512134697992
	variance of the error: 0.22008572968188178
	coefficient w:
[ 0.07100167 -0.12120575  0.0770112  -0.12408284  0.8483982  -0.2015484
 -0.29181564  0.310217   -0.32996821 -0.14397126  0.43087289 -0.41834683
  0.13486752  0.44334455  0.80544847 -0.12614122 -0.20874957  0.02753074
 -0.15818383 -0.38650206  0.72535714  0.43717838 -0.03015154 -0.6686621
  0.526327    0.42239879 -0.12900433  0.27552549 -0.51519828 -0.48234669
 -0.41972959  0.2407297  -0.22531369 -0.39552768 -0.02504781 -0.74832016
 -0.15722596 -0.41669448 -0.15072743  0.78534677 -0.45473953  0.16899104
 -0.26334322  0.34900345  0.04046057  0.9131131  -0.64589075  0.75702978
  0.26542047 -0.18960759  0.15347013  0.81038455 -0.33584148 -0.36222326
  0.1595293  -0.11732606 -0.38988252 -0.03010262  0.63918901 -0.64941979
  0.25105906  0.1494848   0.26922656 -0.11699443  0.42942491 -0.50503928
 -0.11303796 -0.32611895 -0.15499552 -0.00796499

In [30]:
#FRAGILITY - poly
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts, kernel=k1))

Model: RIDGE KERNEL
	regularizer alpha: 9.809999999999999
	mean error: 0.831779043411169
	variance of the error: 0.22162139061697972
	coefficient w:
[ 0.21373178  0.15087671 -0.00156124  0.16795605 -0.03198274  0.00397148
 -0.07640199 -0.11046396  0.09713279 -0.09208949 -0.07015213  0.05631912
  0.02811857 -0.10727042 -0.06239362  0.0425633  -0.05085845  0.03890213
 -0.0512319  -0.04721053 -0.11662321 -0.03142614  0.11939403  0.04668097
 -0.10243868 -0.07447316 -0.08024282  0.12516793  0.04295576 -0.03040095
  0.06434405 -0.03746335  0.1429075   0.14003803  0.07052851 -0.03150858
 -0.02616056  0.0344351  -0.03534399 -0.03695053  0.10723177  0.14738807
  0.01581576  0.19865306  0.15783622 -0.09027462  0.00823604 -0.09324578
 -0.06283044 -0.03998784  0.1398291  -0.10662383  0.17000396  0.00494747
  0.22358106  0.06047907 -0.07394532  0.12406103 -0.07862665 -0.05152998
 -0.11585212  0.08882823  0.11189196  0.07780996  0.11082538  0.11738903
 -0.00081271 -0.04715837  0.04590593 -0.06510209

In [31]:
# FRAGILITY - sigmoid
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, y_f_tr, y_f_ts, kernel=k2))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154

Model: RIDGE KERNEL
	regularizer alpha: 3.51
	mean error: 0.861289535924411
	variance of the error: 0.23803755120141415
	coefficient w:
[ 0.61485873  0.45506025 -0.00464716  0.50964858 -0.30256071 -0.27224061
 -0.23606874 -0.36141447  0.32770513 -0.35576152 -0.29111216  0.40571722
  0.1498559  -0.28096405 -0.18047879  0.11440462 -0.23779906  0.20384434
 -0.18884316 -0.20399816 -0.35003812 -0.10253202  0.33142733  0.31767779
 -0.29105031 -0.24893051 -0.22702966  0.38700505  0.07088803 -0.20597768
  0.16244932 -0.06797073  0.49108725  0.40570196  0.22096556 -0.08100365
 -0.07915323  0.11354237 -0.20701938 -0.28339267  0.37667912  0.45054832
  0.09497616  0.56254487  0.49925689 -0.23030035 -0.12392053 -0.31601314
 -0.17302326 -0.25487492  0.445718   -0.29776464  0.55420053 -0.16276825
  0.61968982  0.16815252 -0.37871729  0.38953715 -0.23851096 -0.142786
 -0.34200757  0.28172706  0.34243166  0.23311617  0.38680766  0.33484242
  0.0551268  -0.12381039  0.08991801 -0.16568227  0.33052087  0

SVM

In [32]:
c={'C':np.arange(1,100,10)}

In [33]:
#LIGHTNESS
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, y_l_tr, y_l_ts))

Model: SVM
	regularizer alpha: 1
	mean error: 0.7020229324548999
	variance of the error: 0.2389987698782022
	coefficient w:
[[-0.71070329 -1.          1.         -0.10466542 -0.86072352  1.
  -1.         -0.16131003  1.         -0.17378382  1.          1.
   1.         -1.         -0.24619375  0.90311709  1.          1.
  -1.         -1.          1.          1.         -1.          1.
  -1.         -0.80110758 -1.          1.         -0.99421288 -1.
  -1.         -1.         -0.06040374 -1.         -0.1139699   1.
  -1.          1.         -0.82764693  1.          1.         -1.
   1.          1.         -0.02303517  0.68266887  1.         -1.
  -0.32708932  1.         -1.         -0.9110996   1.         -1.
   1.          0.2676484   1.         -0.57486845  1.         -1.
  -0.11849375 -1.         -1.         -1.         -1.          1.
   1.         -1.         -1.         -1.          1.          1.
   1.          0.08612763 -1.          1.          1.         -1.
   1.          1. 

In [34]:
#FRAGILITY
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, y_f_tr, y_f_ts))

Model: SVM
	regularizer alpha: 1
	mean error: 0.7994504484927619
	variance of the error: 0.29530427284152166
	coefficient w:
[[ 1.          1.         -1.          1.         -0.28813885  0.00802393
  -1.         -1.          1.         -1.         -1.          1.
   0.27458427 -1.         -1.          0.75850304 -0.45496807  1.
  -0.2371106  -0.64656478 -1.         -1.          1.          1.
  -1.         -1.         -1.          1.          1.         -0.09466275
   1.         -1.          1.          1.          1.         -0.36099741
   1.         -0.65644818  1.          1.          1.          1.
   1.         -1.          0.65973226 -1.         -1.         -0.65590699
   1.         -1.          1.          0.40460926  1.          1.
  -1.          1.         -1.         -0.90778065 -1.          1.
   1.          1.          1.          1.         -1.          0.56588235
  -1.          1.          1.          1.          0.81210002 -1.
  -1.         -0.06301918 -0.08469478 -1.  

RF

In [35]:
#LIGHTNESS
ris = cvu.tree_regression(X_tr, X_ts, y_l_tr, y_l_ts)
print(ris[0], ris[1])
print(ris[2])

0.9842105263157892 0.5144875346260388
[0.03831113 0.00034732 0.01296902 0.         0.         0.
 0.         0.         0.         0.         0.13188249 0.0037047
 0.         0.         0.         0.03005988 0.07227471 0.00266275
 0.         0.00671477 0.00069463 0.         0.         0.
 0.         0.         0.00278624 0.         0.         0.
 0.         0.01642256 0.         0.         0.         0.00138926
 0.00057886 0.         0.00030872 0.00368154 0.         0.
 0.         0.         0.00037047 0.         0.00185235 0.
 0.         0.00023154 0.         0.0008104  0.         0.
 0.00204144 0.00038591 0.         0.01389262 0.04619297 0.
 0.         0.         0.         0.         0.00530713 0.
 0.         0.         0.         0.         0.         0.00216107
 0.         0.         0.02741992 0.00208389 0.         0.
 0.03723274 0.         0.         0.         0.00204144 0.
 0.         0.         0.         0.01237706 0.         0.
 0.0074094  0.         0.         0.         0

In [36]:
#FRAGILITY
ris = cvu.tree_regression(X_tr, X_ts, y_f_tr, y_f_ts)
print(ris[0], ris[1])
print(ris[2])

1.1631578947368424 0.700747922437673
[0.00120971 0.         0.         0.01264331 0.         0.00095452
 0.00187434 0.00157788 0.         0.         0.         0.00017532
 0.         0.00023376 0.         0.         0.         0.
 0.         0.         0.09384617 0.         0.00017532 0.00195775
 0.         0.         0.         0.         0.         0.
 0.14762082 0.         0.         0.00997149 0.         0.
 0.         0.         0.         0.00017532 0.         0.
 0.00017532 0.         0.         0.05471461 0.00078894 0.
 0.         0.         0.         0.         0.         0.01709792
 0.14200974 0.02251895 0.         0.         0.         0.00093504
 0.         0.00023376 0.         0.         0.         0.00052596
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.001461   0.00262981
 0.         0.         0.         0.06947145 0.         0.
 0.         0.00093504 0.00017532 0.         0.01075299 0.02222187
 0.05425899 0.